In [2]:
import pandas as pd
import json
import boto3
import plotly.express as px
from plotly.offline import init_notebook_mode

from urllib.parse import urlparse
import numpy as np
from io import BytesIO
from PIL import Image, ImageDraw, ImageFont

init_notebook_mode(connected=True) 

trip = "91e8ab2b-cc91-419b-9b92-b81feb2ffc6a"

# Query Data

In [3]:
dynamodb_client = boto3.client('dynamodb')
boto3.resource('dynamodb')

result = dynamodb_client.query(
    TableName = 'telematics-images-model-results',
    KeyConditionExpression='trip_id = :trip_id',
    ExpressionAttributeValues={
        ':trip_id': {'S': trip}
    }
)

deserializer = boto3.dynamodb.types.TypeDeserializer()

def transform_item(item):
    data = {k: deserializer.deserialize(v) for k,v in item.items()}
    return data    

my_data = pd.DataFrame.from_dict([transform_item(x) for x in result['Items']])

my_data.head()

,timestamp,model_result,image_path,speed,trip_id,lat,long,accuracy
0,1653957201996,"[{'model_name': 'weather', 'predictions': {'cl...",s3://telematics-images-raw/images/0002523d-1f2...,12.05259322485166,91e8ab2b-cc91-419b-9b92-b81feb2ffc6a,40.47368024924863,-88.95561281273372,4.844246389642948
1,1653958206996,"[{'model_name': 'scene', 'predictions': {'resi...",s3://telematics-images-raw/images/02fc1771-637...,30.14539752431029,91e8ab2b-cc91-419b-9b92-b81feb2ffc6a,40.53850980964621,-88.98091561783319,4.691755378994643
2,1653957501999,"[{'model_name': 'scene', 'predictions': {'resi...",s3://telematics-images-raw/images/031378c0-b52...,6.3935782963526595,91e8ab2b-cc91-419b-9b92-b81feb2ffc6a,40.47371967398804,-88.99319016413831,4.825555737447976
3,1653958016998,"[{'model_name': 'weather', 'predictions': {'cl...",s3://telematics-images-raw/images/034c29a4-022...,12.092980468162018,91e8ab2b-cc91-419b-9b92-b81feb2ffc6a,40.52626978115608,-88.99531524874251,4.899926351533284
4,1653958561997,"[{'model_name': 'scene', 'predictions': {'resi...",s3://telematics-images-raw/images/03a3de16-b21...,17.230094674935536,91e8ab2b-cc91-419b-9b92-b81feb2ffc6a,40.51133508402178,-88.95339315995889,4.82564444291254


In [4]:
my_data['model_result'][250]

[{'model_name': 'scene',
  'predictions': {'residential': Decimal('0.0002217212604591623'),
   'highway': Decimal('0.9863944053649902'),
   'city street': Decimal('0.012242286466062069'),
   'undefined': Decimal('0.0011414935579523444')}},
 {'model_name': 'rekognition_detect_labels',
  'predictions': [{'Instances': [],
    'Confidence': Decimal('98.01627349853516'),
    'Name': 'Highway',
    'Parents': [{'Name': 'Freeway'}, {'Name': 'Road'}]},
   {'Instances': [{'Confidence': Decimal('60.53462219238281'),
      'BoundingBox': {'Height': Decimal('0.3172668516635895'),
       'Left': Decimal('0.0097042853012681'),
       'Top': Decimal('0.6636804938316345'),
       'Width': Decimal('0.5030743479728699')}}],
    'Confidence': Decimal('60.53462219238281'),
    'Name': 'Car',
    'Parents': [{'Name': 'Vehicle'}, {'Name': 'Transportation'}]},
   {'Instances': [],
    'Confidence': Decimal('60.53462219238281'),
    'Name': 'Transportation',
    'Parents': []},
   {'Instances': [],
    'Confi

# Create Features

In [5]:
def count_instances(var, name, detections, new_columns):
   
    instance_list = [x for x in detections if x['Name'] == var]
    
    if instance_list:
        cnt = len(instance_list[0]['Instances'])
        
        # Deals with the case where detections are really a global classification
        if cnt == 0:
            cnt = 1
    else:
        cnt = 0          
    
    new_columns.update({name: cnt}) 

In [6]:
def split_model_results(row):
    x = row['model_result']
    
    prep_variables = (lambda var, new_cols: {f"{var}_{key.replace(' ', '_').replace('/', '_')}": value for key, value in new_cols.items()})
    
    new_columns = {}
    
    # Parse out the model results to a dictionary
    model_results = {'scene': None, 'timeofday': None, 'weather': None, 'rekognition_detect_labels': None}    
    for val in x:
        model_results.update({val['model_name']: val['predictions']})
        
    # Calculate Scene Statistics
    var = 'scene'
    temp = model_results[var].copy()  
    renamed = prep_variables(var, temp)
    new_columns.update(**renamed)
    
    # Calculate Time of Day Statistics
    var = 'timeofday'
    temp = model_results[var].copy()    
    renamed = prep_variables(var, temp)
    new_columns.update(**renamed)
        
    # Calculate Weather Statistics
    var = 'weather'
    temp = model_results[var].copy()  
    temp.update({'dangerous': temp['snowy']+temp['rainy']})
    renamed = prep_variables(var, temp)
    new_columns.update(**renamed)
    
    # Calculate Rekognition Detect Label Statistics
    var = "rekognition_detect_labels"
    temp = model_results[var].copy()
    
    count_instances('Car', 'num_cars', temp, new_columns)    
    count_instances('Traffic Light', 'traffic_light_cnt', temp, new_columns)    
    count_instances('Intersection', 'intersection_ind', temp, new_columns)   
    count_instances('Pedestrian', 'pedestrian_cnt', temp, new_columns)   
    count_instances('Person', 'person_cnt', temp, new_columns)  
    
    new_columns.update({'humans_cnt': max(new_columns['pedestrian_cnt'], new_columns['person_cnt'])})
    
    # Combined Columns
    new_columns.update({'poor_vision': 1 - (1 - new_columns['weather_dangerous']) * (1 - new_columns['timeofday_dawn_dusk'])})
    
    return {k: float(v) for k,v in new_columns.items()}

new_columns = my_data.apply(split_model_results, axis = 1, result_type = 'expand').convert_dtypes(convert_integer = False)
combined_data = my_data.join(new_columns).sort_values('timestamp').reset_index()
combined_data['timestamp'] = pd.to_datetime(combined_data['timestamp'].astype(int), unit='ms').dt.tz_localize('UTC').dt.tz_convert('America/Chicago')
#combined_data['timestamp'] = combined_data['timestamp'].astype(int)
combined_data.head()

,index,timestamp,model_result,image_path,speed,trip_id,lat,long,accuracy,scene_residential,...,weather_partly_cloudy,weather_undefined,weather_dangerous,num_cars,traffic_light_cnt,intersection_ind,pedestrian_cnt,person_cnt,humans_cnt,poor_vision
0,172,2022-05-30 19:27:21.595000-05:00,"[{'model_name': 'scene', 'predictions': {'resi...",s3://telematics-images-raw/images/70b8b439-3a5...,None,91e8ab2b-cc91-419b-9b92-b81feb2ffc6a,40.46495264587518,-88.9263722383015,132.7221758326995,0.020156,...,0.004016,0.756453,0.030260,0.0,0.0,0.0,0.0,0.0,0.0,0.037308
1,42,2022-05-30 19:27:26.806000-05:00,"[{'model_name': 'scene', 'predictions': {'resi...",s3://telematics-images-raw/images/15eae1ba-222...,0.041146863251924515,91e8ab2b-cc91-419b-9b92-b81feb2ffc6a,40.46445702375613,-88.92650844730879,11.45542625561323,0.520593,...,0.312108,0.437144,0.069936,1.0,0.0,0.0,0.0,0.0,0.0,0.950109
2,141,2022-05-30 19:27:31.561000-05:00,"[{'model_name': 'scene', 'predictions': {'resi...",s3://telematics-images-raw/images/59241473-46c...,0.00514473021030426,91e8ab2b-cc91-419b-9b92-b81feb2ffc6a,40.464440029994556,-88.92653468766405,5.26450489620574,0.966057,...,0.577845,0.033049,0.003862,1.0,0.0,0.0,0.0,0.0,0.0,0.881226
3,291,2022-05-30 19:27:37.261000-05:00,"[{'model_name': 'scene', 'predictions': {'resi...",s3://telematics-images-raw/images/bfece4b6-a41...,2.6495614051816783,91e8ab2b-cc91-419b-9b92-b81feb2ffc6a,40.46445916492559,-88.92664009019697,4.740301643263578,0.089008,...,0.146038,0.684568,0.019172,1.0,0.0,0.0,0.0,0.0,0.0,0.909162
4,160,2022-05-30 19:27:46.999000-05:00,"[{'model_name': 'rekognition_detect_labels', '...",s3://telematics-images-raw/images/6a57a96c-197...,7.590029716486336,91e8ab2b-cc91-419b-9b92-b81feb2ffc6a,40.464665912144,-88.92742441191201,4.890624189348712,0.872508,...,0.489269,0.330733,0.012510,1.0,0.0,0.0,0.0,0.0,0.0,0.870180


# Trip Level Summary

In [7]:
combined_data.groupby('trip_id').agg('mean').drop('index', axis = 1).T

trip_id,91e8ab2b-cc91-419b-9b92-b81feb2ffc6a
scene_residential,0.063580
scene_highway,0.630427
scene_city_street,0.232002
scene_undefined,0.073992
timeofday_dawn_dusk,0.689693
timeofday_daytime,0.294873
timeofday_night,0.006560
timeofday_undefined,0.008874
weather_clear,0.266123
weather_rainy,0.006408


# Visualize

In [8]:
fig = px.scatter(combined_data, 
                 x='timestamp', 
                 y=["timeofday_daytime", "timeofday_dawn_dusk"],  
                 trendline="lowess", trendline_options=dict(frac=0.05))
fig.data = [t for t in fig.data if t.mode == "lines"]
fig.update_traces(showlegend=True)

# Turned Onto Vet
fig.add_vrect(
    x0="2022-05-30 19:51:37.261000-05:00", x1="2022-05-30 20:00:37.261000-05:10",
    fillcolor="Grey", opacity=0.25,
    annotation_text="Turned South on Veterans Parkway", annotation_position="top left",
    layer="below", line_width=0,
)

# Entered Downtown
fig.add_vrect(
    x0="2022-05-30 19:38:37.261000-05:00", x1="2022-05-30 19:42:37.261000-05:10",
    annotation_text="Entered Downtown", annotation_position="top left",
    fillcolor="Grey", opacity=0.25,
    layer="below", line_width=0,
)

fig.write_html('trip_visuals/timeseries_daytime.html', auto_open=True)

In [9]:
fig = px.scatter(combined_data, x='timestamp', y=["weather_clear", "weather_partly_cloudy", "weather_overcast", "weather_rainy", "weather_snowy"],  
                 trendline="lowess", trendline_options=dict(frac=0.05))
fig.data = [t for t in fig.data if t.mode == "lines"]
fig.update_traces(showlegend=True)

# Turned Onto Vet
fig.add_vrect(
    x0="2022-05-30 19:51:37.261000-05:00", x1="2022-05-30 20:00:37.261000-05:10",
    fillcolor="Grey", opacity=0.25,
    annotation_text="Turned South on Veterans Parkway", annotation_position="top left",
    layer="below", line_width=0,
)

# Entered Downtown
fig.add_vrect(
    x0="2022-05-30 19:38:37.261000-05:00", x1="2022-05-30 19:42:37.261000-05:10",
    annotation_text="Entered Downtown", annotation_position="top left",
    fillcolor="Grey", opacity=0.25,
    layer="below", line_width=0,
)

fig.write_html('trip_visuals/timeseries_weather.html', auto_open=True)

In [10]:
fig = px.scatter(combined_data, 
                 x='timestamp', 
                 y=["num_cars"],  
                 trendline="lowess", 
                 trendline_options=dict(frac=0.05))
fig.data = [t for t in fig.data if t.mode == "lines"]
fig.update_traces(showlegend=True)

# Turned Onto Vet
fig.add_vrect(
    x0="2022-05-30 19:51:37.261000-05:00", x1="2022-05-30 20:00:37.261000-05:10",
    fillcolor="Grey", opacity=0.25,
    annotation_text="Turned South on Veterans Parkway", annotation_position="top left",
    layer="below", line_width=0,
)

# Entered Downtown
fig.add_vrect(
    x0="2022-05-30 19:38:37.261000-05:00", x1="2022-05-30 19:42:37.261000-05:10",
    annotation_text="Entered Downtown", annotation_position="top left",
    fillcolor="Grey", opacity=0.25,
    layer="below", line_width=0,
)

fig.write_html('trip_visuals/timeseries_num_cars.html', auto_open=True)

In [11]:
fig = px.line_mapbox(combined_data, lat = "lat", lon = "long", mapbox_style = 'stamen-toner', hover_data=["timestamp"], zoom = 12)
fig.update_traces(line=dict(color="Red", width=5))
fig.write_html('trip_visuals/general_map.html', auto_open=True)

In [12]:
keep = combined_data['traffic_light_cnt'] > 0
temp_df = combined_data[keep]

fig = px.scatter_mapbox(temp_df, lat = "lat", lon = "long", mapbox_style = 'stamen-toner', zoom = 12)
fig.update_traces(marker=dict(size=15))
fig.write_html('trip_visuals/traffic_light_map.html', auto_open=True)

In [13]:
keep = combined_data['intersection_ind'] > 0
temp_df = combined_data[keep]

fig = px.scatter_mapbox(temp_df, lat = "lat", lon = "long", mapbox_style = 'stamen-toner', zoom = 12)
fig.update_traces(marker=dict(size=15))
fig.write_html('trip_visuals/intersection_map.html', auto_open=True)

In [14]:
keep = combined_data['scene_city_street'] > 0.5
temp_df = combined_data[keep]

fig = px.scatter_mapbox(temp_df, lat = "lat", lon = "long", mapbox_style = 'stamen-toner', zoom = 12)
fig.update_traces(marker=dict(size=15))
fig.write_html('trip_visuals/road_type_map.html', auto_open=True)

# Animated GIF Creation

In [15]:
# Demos what capturing an image every 5 frames looks like

temp_df = combined_data
s3 = boto3.resource('s3', region_name='us-east-1')

images = []
for i, row in temp_df[150:175].iterrows():
    
    s3_path = row['image_path']
    o = urlparse(s3_path, allow_fragments=False)
    bucket = s3.Bucket(o.netloc)
    object = bucket.Object(o.path[1:])
    img_data = object.get().get('Body').read()
    img = Image.open(BytesIO(img_data))
    
    images.append(img)
    
images[0].save("trip_visuals/data_capture_demo.gif", save_all=True, append_images=images[1:], duration=1000, loop=0)

In [16]:
# Demos a single image capture with many classifications

row = temp_df.iloc[250]
s3_path = row['image_path']
o = urlparse(s3_path, allow_fragments=False)
bucket = s3.Bucket(o.netloc)
object = bucket.Object(o.path[1:])
img_data = object.get().get('Body').read()
img = Image.open(BytesIO(img_data))
    
I1 = ImageDraw.Draw(img)
    
myFont = ImageFont.truetype('RobotoMono-Bold.ttf', 15)

var = 'weather'
columns = [x for x in row.index.tolist() if var in x]
max_col = pd.to_numeric(row[columns]).idxmax()
max_col_pretty = max_col.replace(f"{var}_", "").replace("_", " ").title()
max_p = row[max_col]
weather_string = f"{max_col_pretty} - {max_p:.1%}"

var = 'timeofday'
columns = [x for x in row.index.tolist() if var in x]
max_col = pd.to_numeric(row[columns]).idxmax()
max_col_pretty = max_col.replace(f"{var}_", "").replace("_", " ").title()
max_p = row[max_col]
timeofday_string = f"{max_col_pretty} - {max_p:.1%}"


var = 'scene'
columns = [x for x in row.index.tolist() if var in x]
max_col = pd.to_numeric(row[columns]).idxmax()
max_col_pretty = max_col.replace(f"{var}_", "").replace("_", " ").title()
max_p = row[max_col]
scene_string = f"{max_col_pretty} - {max_p:.1%}"


message = f"Number of Cars: {row['num_cars']}\nNear an Intersection: {row['intersection_ind']}\n{weather_string}\n{timeofday_string}\n{scene_string}"
    
shape = [(0, 360), (250, 480)]
I1.rectangle(shape, fill ="#000000")
I1.multiline_text((10, 360), message, font = myFont, fill=(255, 0, 0))
    
img.save("trip_visuals/single_image_labels.png")

In [17]:
# Demo's number of cars over time

temp_df = combined_data
s3 = boto3.resource('s3', region_name='us-east-1')

images = []
for i, row in temp_df.iterrows():
    if i % 10 != 0:
        continue
    
    s3_path = row['image_path']
    o = urlparse(s3_path, allow_fragments=False)
    bucket = s3.Bucket(o.netloc)
    object = bucket.Object(o.path[1:])
    img_data = object.get().get('Body').read()
    img = Image.open(BytesIO(img_data))
    
    I1 = ImageDraw.Draw(img)
    
    myFont = ImageFont.truetype('RobotoMono-Bold.ttf', 15)
    message = f"Number of Cars: {row['num_cars']}\nNear an Intersection: {row['intersection_ind']}"
    
    shape = [(0, 440), (250, 480)]
    I1.rectangle(shape, fill ="#000000")
    I1.multiline_text((10, 440), message, font = myFont, fill=(255, 0, 0))
    
    images.append(img)
    
images[0].save("trip_visuals/number_of_cars_demo.gif", save_all=True, append_images=images[1:], duration=1000, loop=0)

In [22]:
# Demos weather over time

temp_df = combined_data
s3 = boto3.resource('s3', region_name='us-east-1')

images = []
for i, row in temp_df.iterrows():
    if i % 10 != 0:
        continue
    
    s3_path = row['image_path']
    o = urlparse(s3_path, allow_fragments=False)
    bucket = s3.Bucket(o.netloc)
    object = bucket.Object(o.path[1:])
    img_data = object.get().get('Body').read()
    img = Image.open(BytesIO(img_data))
    
    I1 = ImageDraw.Draw(img)
    
    myFont = ImageFont.truetype('RobotoMono-Bold.ttf', 10)    
      
    bar_thickness = 10
    max_length = 100
    offset_y = 480 - 6 * bar_thickness
    offset_x = 100  
    
    # Black Background
    shape = [(0, offset_y - bar_thickness), (offset_x + max_length, 480)]
    I1.rectangle(shape, fill ="#000000")
    
    shape = [(offset_x, offset_y), (offset_x + max_length * row['weather_clear'], offset_y + bar_thickness - 1)]
    I1.rectangle(shape, fill ="#fde725FF")
    I1.text((5, offset_y), 'Clear', font = myFont, fill=(255, 255, 255))
    
    shape = [(offset_x, offset_y + bar_thickness), (offset_x + max_length * row['weather_partly_cloudy'], offset_y + 2 * bar_thickness - 1)]
    I1.rectangle(shape, fill ="#5ec962")
    I1.text((5, offset_y + bar_thickness), 'Partly Cloudy', font = myFont, fill=(255, 255, 255))
    
    shape = [(offset_x, offset_y + 2 * bar_thickness), (offset_x + max_length * row['weather_overcast'], offset_y + 3 * bar_thickness - 1)]
    I1.rectangle(shape, fill ="#21918c")
    I1.text((5, offset_y + 2 * bar_thickness), 'Overcast', font = myFont, fill=(255, 255, 255))

    combined = row['weather_undefined'] + row['weather_rainy'] + row['weather_snowy']
    shape = [(offset_x, offset_y + 3 * bar_thickness), (offset_x + max_length * combined, offset_y + 4 * bar_thickness - 1)]
    I1.rectangle(shape, fill ="#3b528b")
    I1.text((5, offset_y + 3 * bar_thickness), 'Other', font = myFont, fill=(255, 255, 255))
    
    images.append(img)
    
images[0].save("trip_visuals/weather_demo.gif", save_all=True, append_images=images[1:], duration=1000, loop=0)

In [19]:
# Demos Time of Day Over Time

temp_df = combined_data
s3 = boto3.resource('s3', region_name='us-east-1')

images = []
for i, row in temp_df.iterrows():
    if i % 10 != 0:
        continue
    
    s3_path = row['image_path']
    o = urlparse(s3_path, allow_fragments=False)
    bucket = s3.Bucket(o.netloc)
    object = bucket.Object(o.path[1:])
    img_data = object.get().get('Body').read()
    img = Image.open(BytesIO(img_data))
    
    I1 = ImageDraw.Draw(img)
    
    myFont = ImageFont.truetype('RobotoMono-Bold.ttf', 10)    
      
    bar_thickness = 10
    max_length = 100
    offset_y = 480 - 6 * bar_thickness
    offset_x = 75  
    
    # Black Background
    shape = [(0, offset_y - bar_thickness), (offset_x + max_length, 480)]
    I1.rectangle(shape, fill ="#000000")
    
    # Highway Bar
    shape = [(offset_x, offset_y), (offset_x + max_length * row['timeofday_daytime'], offset_y + bar_thickness - 1)]
    I1.rectangle(shape, fill ="#fde725FF")
    I1.text((5, offset_y), 'Day', font = myFont, fill=(255, 255, 255))
    
    # City Street Bar
    shape = [(offset_x, offset_y + bar_thickness), (offset_x + max_length * row['timeofday_dawn_dusk'], offset_y + 2 * bar_thickness - 1)]
    I1.rectangle(shape, fill ="#5ec962")
    I1.text((5, offset_y + bar_thickness), 'Dawn/Dusk', font = myFont, fill=(255, 255, 255))
    
    # Residential Bar
    shape = [(offset_x, offset_y + 2 * bar_thickness), (offset_x + max_length * row['timeofday_night'], offset_y + 3 * bar_thickness - 1)]
    I1.rectangle(shape, fill ="#21918c")
    I1.text((5, offset_y + 2 * bar_thickness), 'Night', font = myFont, fill=(255, 255, 255))
    
    # Other Bar
    shape = [(offset_x, offset_y + 3 * bar_thickness), (offset_x + max_length * row['timeofday_undefined'], offset_y + 4 * bar_thickness - 1)]
    I1.rectangle(shape, fill ="#3b528b")
    I1.text((5, offset_y + 3 * bar_thickness), 'Other', font = myFont, fill=(255, 255, 255))
    
    images.append(img)
    
images[0].save("trip_visuals/timeofday_demo.gif", save_all=True, append_images=images[1:], duration=1000, loop=0)

In [20]:
# Demos Scene Over Time

temp_df = combined_data
s3 = boto3.resource('s3', region_name='us-east-1')

images = []
for i, row in temp_df.iterrows():
    if i % 10 != 0:
        continue
    
    s3_path = row['image_path']
    o = urlparse(s3_path, allow_fragments=False)
    bucket = s3.Bucket(o.netloc)
    object = bucket.Object(o.path[1:])
    img_data = object.get().get('Body').read()
    img = Image.open(BytesIO(img_data))
    
    I1 = ImageDraw.Draw(img)
    
    myFont = ImageFont.truetype('RobotoMono-Bold.ttf', 10)    
      
    bar_thickness = 10
    max_length = 100
    offset_y = 480 - 6 * bar_thickness
    offset_x = 75  
    
    # Black Background
    shape = [(0, offset_y - bar_thickness), (offset_x + max_length, 480)]
    I1.rectangle(shape, fill ="#000000")
    
    # Highway Bar
    shape = [(offset_x, offset_y), (offset_x + max_length * row['scene_highway'], offset_y + bar_thickness - 1)]
    I1.rectangle(shape, fill ="#fde725FF")
    I1.text((5, offset_y), 'Highway', font = myFont, fill=(255, 255, 255))
    
    # City Street Bar
    shape = [(offset_x, offset_y + bar_thickness), (offset_x + max_length * row['scene_city_street'], offset_y + 2 * bar_thickness - 1)]
    I1.rectangle(shape, fill ="#5ec962")
    I1.text((5, offset_y + bar_thickness), 'Street', font = myFont, fill=(255, 255, 255))
    
    # Residential Bar
    shape = [(offset_x, offset_y + 2 * bar_thickness), (offset_x + max_length * row['scene_residential'], offset_y + 3 * bar_thickness - 1)]
    I1.rectangle(shape, fill ="#21918c")
    I1.text((5, offset_y + 2 * bar_thickness), 'Residential', font = myFont, fill=(255, 255, 255))
    
    # Other Bar
    shape = [(offset_x, offset_y + 3 * bar_thickness), (offset_x + max_length * row['scene_undefined'], offset_y + 4 * bar_thickness - 1)]
    I1.rectangle(shape, fill ="#3b528b")
    I1.text((5, offset_y + 3 * bar_thickness), 'Other', font = myFont, fill=(255, 255, 255))
    
    images.append(img)
    
images[0].save("trip_visuals/scene_demo.gif", save_all=True, append_images=images[1:], duration=1000, loop=0, optimize=False)